In [7]:
import os, sys, email
import numpy as np
import pandas as pd
import nltk
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [ ]:
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "wiki-research-l/2020-July"
file_list = getFileList(directory_path)
file_list.sort()

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [ ]:
# メールのBody部分を各パートに分解する
bodies = {dict(Bodies=[])}
for idx, body in mail_df.loc[:,'Body'].items():
    origin = []
    greetings = dict(Greetings=[])
    sentences = dict(Sentence=[])
    captions = dict(Caption=[])
    bulletlist = dict(Bulletlist=[])
    ending = dict(Ending=[])
    quotation = dict(Quotation=[])
    footer = dict(Footer=[])
    misc = dict(Misc=[])

    lines = body.splitlines()
    for num, line in enumerate(lines):
        if re.match(r'\[G\]',line) is not None:
            greetings['Greetings'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[S\]',line) is not None:
            sentences['Sentence'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[C\]',line) is not None:
            captions['Caption'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[B\]',line) is not None:
            bulletlist['Bulletlist'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[E\]',line) is not None:
            ending['Ending'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[Q\]',line) is not None:
            quotation['Quotation'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[F\]',line) is not None:
            footer['Footer'].append({num:line[3:]})
            origin.append(line[3:])
        elif re.match(r'\[M\]',line) is not None:
            misc['Misc'].append({num:line[3:]})
            origin.append(line[3:])
        else: #空白行に対応する
            continue
    originbody = ' '.join(origin)
    bodies['Bodies'].append({'idx':idx, 'countrows':len(lines), 'body':originbody, **greetings, **sentences, **captions, **bulletlist, **ending, **quotation, **footer, **misc})

In [ ]:
#　bodiesのsentenceのvalueから複数文のテキストを作る
s_list = []
for i in bodies['Bodies']:  #1通ずつ取り出す
    text = ''
    for j in i['Sentence']: #1行{行番号:文}ずつ取り出し，複数文が含まれた1つの文字列にする
        text = text + list(j.values())[0] + ' '
    s_list.append(text) #1通ごとの自然文のテキストをリストに格納する

In [ ]:
# センテンスのdataframeを作る
sentence_list = []
# 文章全体に対する前処理
for i, content in enumerate(s_list):
    sentences = tokenize.sent_tokenize(content)
    # 文に対する前処理
    for j, sentence in enumerate(sentences):
        sentence = re.sub(r'\s{2,}',' ',sentence)
        sentence_list.append([mail_df['Message-ID'][i],sentence])
sentence_df = pd.DataFrame(sentence_list,columns=['Message-ID','sentence'])

In [4]:
#sentence_df.to_csv('wiki-research-l/output/sentence_df.csv')
sentence_df = pd.read_csv('/Users/taroaso/myprojects/OpenIE/wiki-research-l/output/sentence_df.csv',index_col=0)
sentence_df.head()

,Message-ID,sentence
0,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,Thanks for your reply!
1,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,It's a great suggestion to include a list of ORES based tools for people who are not familiar with ORES itself.
2,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,"For our research, we would love to interview you and test our visualizations."
3,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,Please let me know what times will work best for you so we can schedule a meeting.
4,<CAE4fJj_b7k8yeqmz19a-seo3gtL9Mg1nvQO0oPUtrxvCFMkEaw@mail.gmail.com>,Thanks for clarifying.


In [82]:
words = [nltk.word_tokenize(sent) for sent in sentence_df['sentence']]
pos_words = [nltk.pos_tag(word) for word in words]
chunk_words = [nltk.chunk.ne_chunk(pos_word, binary=True) for pos_word in pos_words]
print(pos_words[1])

[('It', 'PRP'), ("'s", 'VBZ'), ('a', 'DT'), ('great', 'JJ'), ('suggestion', 'NN'), ('to', 'TO'), ('include', 'VB'), ('a', 'DT'), ('list', 'NN'), ('of', 'IN'), ('ORES', 'NNP'), ('based', 'VBN'), ('tools', 'NNS'), ('for', 'IN'), ('people', 'NNS'), ('who', 'WP'), ('are', 'VBP'), ('not', 'RB'), ('familiar', 'JJ'), ('with', 'IN'), ('ORES', 'NNP'), ('itself', 'PRP'), ('.', '.')]


In [117]:
pos_words[11]

[('CAT', 'NNP'),
 ('Lab', 'NNP'),
 ('<', 'NNP'),
 ('https', 'NN'),
 (':', ':'),
 ('//citizensandtech.org/', 'NN'),
 ('>', 'NN'),
 ('has', 'VBZ'),
 ('partnered', 'VBN'),
 ('with', 'IN'),
 ('Arabic', 'NNP'),
 (',', ','),
 ('German', 'NNP'),
 (',', ','),
 ('Persian', 'NNP'),
 ('and', 'CC'),
 ('Polish', 'JJ'),
 ('language', 'NN'),
 ('Wikipedias', 'NNP'),
 ('to', 'TO'),
 ('answer', 'VB'),
 ('those', 'DT'),
 ('questions', 'NNS'),
 ('-', ':'),
 ('and', 'CC'),
 ('more', 'JJR'),
 ('-', ':'),
 ('in', 'IN'),
 ('two', 'CD'),
 ('new', 'JJ'),
 ('studies', 'NNS'),
 ('.', '.')]

In [100]:
rel = nltk.word_tokenize('are not')
arg2s = nltk.word_tokenize('familiar with ORES')
#test = [nltk.pos_tag(t) for t in test] 
#test = [nltk.chunk.ne_chunk(t, binary=True) for t in test]

cnt = 0
for word in arg2s:
    idx = words[1].index(word)
    pos = pos_words[1][idx][1]
    if pos in ('IN', 'TO', 'JJ'):
        rel.append(word)
        cnt = cnt + 1
    else:
        break
del arg2s[:cnt]
rel = ' '.join(rel)
arg2s = ' '.join(arg2s)
print(rel)
print(arg2s)

are not familiar with
ORES


In [119]:
for message_id, sentence, arg1, rel, arg2s in zip(kb_df['message_id'],kb_df['sentence'],kb_df['arg1'],kb_df['rel'],kb_df['arg2s']):
    chunk = nltk.word_tokenize(arg2s)
    chunk = nltk.pos_tag(chunk)
    print(list(map(lambda x: x[0], chunk)), list(map(lambda x: x[1], chunk)))

['familiar', 'with', 'ORES'] ['JJ', 'IN', 'NNP']
['a', 'list', 'of', 'ORES', 'based', 'tools', 'for', 'people'] ['DT', 'NN', 'IN', 'NNP', 'VBN', 'NNS', 'IN', 'NNS']
['a', 'great', 'suggestion', 'to', 'include', 'a', 'list', 'of', 'ORES', 'based', 'tools', 'for', 'people'] ['DT', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'VBN', 'NNS', 'IN', 'NNS']
['our', 'visualizations'] ['PRP$', 'NNS']
['to', 'test', 'our', 'visualizations', 'For', 'our', 'research'] ['TO', 'VB', 'PRP$', 'NNS', 'IN', 'PRP$', 'NN']
['you'] ['PRP']
['to', 'interview', 'you', 'For', 'our', 'research'] ['TO', 'NN', 'PRP', 'IN', 'PRP$', 'NN']
['a', 'meeting'] ['DT', 'NN']
['familiar', 'with', 'ORES'] ['JJ', 'IN', 'NNP']
['for', 'developers', 'in', 'this', 'research'] ['IN', 'NNS', 'IN', 'DT', 'NN']
['ORES'] ['NN']
['familiar', 'with', 'ORES'] ['JJ', 'IN', 'NNP']
['for', 'researchers', 'in', 'this', 'research'] ['IN', 'NNS', 'IN', 'DT', 'NN']
['to', 'bring', 'together', 'all', 'parties', 'now', 'in', 'its', 'second'

TypeError: expected string or bytes-like object

In [49]:
type(sentences[1])

nltk.tree.Tree

In [50]:
current_chunk = []
continuous_chunk = []

for subtree in sentences[1]:
    if type(subtree) == nltk.tree.Tree:
        current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
    elif current_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)
            current_chunk = []
    else:
        continue

continuous_chunk

['ORES']

In [22]:
kb_df = pd.read_csv('/Users/taroaso/myprojects/OpenIE/wiki-research-l/output/full_replaced_triple_from_text_part.csv',index_col=0)
kb_df.head()

,message_id,sentence,arg1,rel,arg2s,confidence,new_arg1,new_rel,new_arg2s
0,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,It's a great suggestion to include a list of ORES based tools for people who are not familiar with ORES itself.,people,are not,familiar with ORES,0.895843,people,are not,familiar with ORES
1,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,It's a great suggestion to include a list of ORES based tools for people who are not familiar with ORES itself.,a great suggestion,to include,a list of ORES based tools for people,0.925061,a great suggestion,to include,a list of ORES based tools for people
2,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,It's a great suggestion to include a list of ORES based tools for people who are not familiar with ORES itself.,It,'s,a great suggestion to include a list of ORES based tools for people,0.678369,It,'s,a great suggestion to include a list of ORES based tools for people
3,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,"For our research, we would love to interview you and test our visualizations.",we,would love to test,our visualizations,0.256095,Ethan Ye,would love to test,Ethan Ye's visualizations
4,<CAE4fJj-un1Um+3aE1jTe9b8WQZuFLMaaFmCJ9zNtzTkuUja0Rw@mail.gmail.com>,"For our research, we would love to interview you and test our visualizations.",we,would love,to test our visualizations For our research,0.380779,Ethan Ye,would love,to test Ethan Ye's visualizations For Ethan Ye's research


In [ ]:
# OpenIEにかける

from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for i,sentence in sentence_df['sentence'].items():
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

In [ ]:
# OIEの抽出結果を整形する

rows = []
for i, extractions in enumerate(extractions_list):
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            message_id = sentence_df['Message-ID'][i]
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [message_id, sentence, arg1, rel, arg2s, confidence]
            rows.append(row)

# 整形結果をdataframeにする
kb_df = pd.DataFrame(rows, columns = ['message_id','sentence', 'arg1', 'rel', 'arg2s', 'confidence'])
kb_df.head()
# dataframeをcsv出力する
kb_df.to_csv('wiki-research-l/output/triple_from_text_part.csv')

# dataframeをRDBのテーブルにする
#from db import connect
#engine = connect()
#kb_df.to_sql(name='kb_wiki_research_l_text',con=engine,if_exists='replace',index=None)

In [ ]:
# triple中の代名詞youの候補の辞書を作るために，Greetingsの行からYouの候補を取り出す．

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

you = {}
for mail in bodies['Bodies']:
    idx = mail['idx']
    for greetings in mail['Greetings']:
        doc = nlp(greetings[0])
        you[idx] = [(X.text, X.label_) for X in doc.ents if X.label_ not in ['DATE','TIME','PERCENT','MONEY','QUANTITY','ORDINAL','CARDINAL']]
        if you[idx] == []:
            del you[idx]

In [ ]:
# tripleのI, MY, ME, WE, OUR, USの代名詞をSenderに置き換えるための辞書
refer_you = {}
for idx, candidate in you.items():
    message_id = mail_df["Message-ID"][idx]
    refer_you[message_id] = {"YOU":candidate[0][0], "YOUR":candidate[0][0] + '\'s'}

# tripleのI, MY, ME, WE, OUR, USの代名詞をSenderに置き換えるための辞書
refer_pronoun = {}
for row in kb_df.values:
    message_id = row[0]
    sender = mail_df[mail_df['Message-ID']==message_id]['From'].values[0]
    start = re.search(r'(\(.+\))',sender).start()
    end = re.search(r'(\(.+\))',sender).end()
    sender = sender[start+1:end-1]
    refer_pronoun[message_id]={'I':sender, 'MY':sender + '\'s', 'ME':sender, 'WE':sender, 'OUR':sender + '\'s', 'US':sender}

# 2つの辞書を結合する
for key, value in refer_you.items():
    if key in refer_pronoun:
        d = refer_pronoun[key]
        d.update(value)
        refer_pronoun[key] = d
    else:
        refer_pronoun[key] = value

In [ ]:
# Senderに置き換えるための辞書を使って実際に置き換える
replaced_rows = []
for row in kb_df.values:
    message_id = row[0]
    arg1 = row[2].split()
    rel = row[3].split()
    arg2s = row[4].split()
    replaced = []
    # arg1の置き換え
    for i, word in enumerate(arg1):
        sender = refer_pronoun[message_id].get(word.upper())
        if sender is None:
            continue
        else:
            arg1[i] = sender
    new_arg1 = ' '.join(arg1)
    replaced.append(new_arg1)
    # relの置き換え
    for i, word in enumerate(rel):
        sender = refer_pronoun[message_id].get(word.upper())
        if sender is None:
            continue
        else:
            rel[i] = sender
    new_rel = ' '.join(rel)
    replaced.append(new_rel)
    # arg2sの置き換え
    for i, word in enumerate(arg2s):
        sender = refer_pronoun[message_id].get(word.upper())
        if sender is None:
            continue
        else:
            arg2s[i] = sender
    new_arg2s = ' '.join(arg2s)
    replaced.append(new_arg2s)
    # [new_arg1, new_rel, new_arg2s]を1行として追加
    replaced_rows.append(replaced)

In [ ]:
replaced_triples = pd.DataFrame(replaced_rows,columns=['new_arg1','new_rel','new_arg2s'])
kb_df = pd.concat([kb_df, replaced_triples],axis=1)
kb_df.to_csv('/Users/taroaso/myprojects/OpenIE/wiki-research-l/output/full_replaced_triple_from_text_part.csv')

In [ ]:
from entityLinking import tagme, confidentAnnotations, mediaWiki

entity_list = []
for i, sentence in sentence_df['sentence'].items():
    json_res = tagme(sentence)
    for candidate in json_res['annotations']:
        if candidate['rho'] >= 0.3:
            entity_list.append([i, candidate['spot'],candidate['rho'],candidate['id'],candidate['title']])
        else:
            continue

# 抽出したentityをdataframeに格納する
rows = []
for row in entity_list:
    rows.append([sentence_df['Message-ID'][row[0]], row[1], row[2], row[3], row[4]])
entity_df = pd.DataFrame(rows,columns=['Message-ID','spot','rho','id','title'])
entity_df.head()

In [ ]:
#entity_df = pd.read_csv('wiki-research-l/output/entity_df.csv', index_col=0)
entity_df[entity_df['rho'] < 0.3][0:100]

In [ ]:
kb_df.head()

In [ ]:
from extraction import entityExtraction

ner_list = []
for i, sentence in sentence_df['sentence'].items():
    entities = entityExtraction(sentence)
    ner_list.append(entities)
len(ner_list)